In [20]:
import pandas as pd  
import numpy as np 

In [21]:
dat = pd.read_csv("used_car_dataset.csv", index_col=0)

In [22]:
dat.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 60109 entries, 0 to 60108
Data columns (total 11 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   brand                  60109 non-null  object 
 1   currency               60109 non-null  object 
 2   description            60109 non-null  object 
 3   fuel_type              60109 non-null  object 
 4   item_condition         60109 non-null  object 
 5   manufacturer           60109 non-null  object 
 6   mileage_from_odometer  60109 non-null  object 
 7   model_date             60109 non-null  float64
 8   price                  60109 non-null  float64
 9   vehicle_engine         60109 non-null  object 
 10  vehicle_transmission   60109 non-null  object 
dtypes: float64(2), object(9)
memory usage: 5.5+ MB


In [23]:
dat.head()

,brand,currency,description,fuel_type,item_condition,manufacturer,mileage_from_odometer,model_date,price,vehicle_engine,vehicle_transmission
0,MG,PKR,MG HS 2021 for sale in Lahore,Petrol,used,MG,"6,000 km",2021.0,6525000.0,1490cc,Automatic
1,Toyota,PKR,Toyota Aygo 2012 for sale in Islamabad,Petrol,used,Toyota,"68,000 km",2012.0,1625000.0,1000cc,Manual
2,Honda,PKR,Honda City 2015 for sale in Lahore,Petrol,used,Honda,"150,000 km",2015.0,2550000.0,1300cc,Automatic
3,KIA,PKR,KIA Sorento 2022 for sale in Sialkot,Petrol,used,KIA,"18,000 km",2022.0,9000000.0,3500cc,Automatic
4,Toyota,PKR,Toyota Corolla 2015 for sale in Rawalpindi,Petrol,used,Toyota,"110,000 km",2015.0,3350000.0,1300cc,Automatic


1. cleaning
  * drop missing values when feature extracting

2. feature eng from description
  - brand
  - carname 
  - location
  - year made


In [24]:
#remove duplicated column
dat["brand"].rename("brand(manufacturer)", inplace=True)
dat.drop("manufacturer", axis="columns", inplace=True)

In [25]:
#extract features in description
dat["description"].str.extract(r'(\w+) (\w+ )(\d+) for sale in (\w+)')

,0,1,2,3
0,MG,HS,2021,Lahore
1,Toyota,Aygo,2012,Islamabad
2,Honda,City,2015,Lahore
3,KIA,Sorento,2022,Sialkot
4,Toyota,Corolla,2015,Rawalpindi
...,...,...,...,...
60104,KIA,Sportage,2022,Lahore
60105,KIA,Picanto,2022,Lahore
60106,Suzuki,Swift,2022,Lahore
60107,Wagon,R,2018,Rawalpindi


In [26]:
#create new features from description and delete description
dat["Car_name"] = dat["description"].str.extract(r'(\w+) (\w+ )(\d+) for sale in (\w+)').get(1)
dat["Year"] = dat["description"].str.extract(r'(\w+) (\w+ )(\d+) for sale in (\w+)').get(2).dropna()
dat["Location"] = dat["description"].str.extract(r'(\w+) (\w+ )(\d+) for sale in (\w+)').get(3)
dat.drop("description", axis="columns", inplace=True)

In [27]:
#drop missing values after new features added 
dat.dropna(inplace=True)

In [28]:
dat.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 58950 entries, 0 to 60108
Data columns (total 12 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   brand                  58950 non-null  object 
 1   currency               58950 non-null  object 
 2   fuel_type              58950 non-null  object 
 3   item_condition         58950 non-null  object 
 4   mileage_from_odometer  58950 non-null  object 
 5   model_date             58950 non-null  float64
 6   price                  58950 non-null  float64
 7   vehicle_engine         58950 non-null  object 
 8   vehicle_transmission   58950 non-null  object 
 9   Car_name               58950 non-null  object 
 10  Year                   58950 non-null  object 
 11  Location               58950 non-null  object 
dtypes: float64(2), object(10)
memory usage: 5.8+ MB


In [29]:
#optional
dat["mileage_from_odometer_km"] = dat["mileage_from_odometer"].str.extract(r'(\d+,\d+) (\w+)').dropna().get(0).apply(lambda x: int(x.replace(',', '')))
dat.drop("mileage_from_odometer", axis="columns", inplace=True)

dat["vehicle_engine_cc"] = dat["vehicle_engine"].str.extract(r'(\d+)').get(0).dropna().apply(lambda x: int(x))
dat.drop("vehicle_engine", axis="columns", inplace=True)

dat.dropna(inplace=True)

In [30]:
#convert "PKR" currency to "USD"
dat["price_usd"] = dat["price"].apply(lambda x: round(x/282.01, ndigits=2))
dat.drop(["price", "currency"], axis="columns", inplace=True)

In [31]:
dat

,brand,fuel_type,item_condition,model_date,vehicle_transmission,Car_name,Year,Location,mileage_from_odometer_km,vehicle_engine_cc,price_usd
0,MG,Petrol,used,2021.0,Automatic,HS,2021,Lahore,6000.0,1490.0,23137.48
1,Toyota,Petrol,used,2012.0,Manual,Aygo,2012,Islamabad,68000.0,1000.0,5762.21
2,Honda,Petrol,used,2015.0,Automatic,City,2015,Lahore,150000.0,1300.0,9042.23
3,KIA,Petrol,used,2022.0,Automatic,Sorento,2022,Sialkot,18000.0,3500.0,31913.76
4,Toyota,Petrol,used,2015.0,Automatic,Corolla,2015,Rawalpindi,110000.0,1300.0,11879.01
...,...,...,...,...,...,...,...,...,...,...,...
60090,Suzuki,Petrol,used,2021.0,Automatic,Sierra,2021,Lahore,7000.0,660.0,17020.67
60093,Honda,Petrol,used,2020.0,Automatic,Accord,2020,Lahore,25000.0,1500.0,53189.60
60103,Lexus,Petrol,used,2003.0,Automatic,Series,2003,Lahore,80792.0,4700.0,27658.59
60107,Suzuki,Hybrid,used,2018.0,Automatic,R,2018,Rawalpindi,15000.0,660.0,11258.47


In [32]:
dat.item_condition.unique()

array(['used'], dtype=object)

In [33]:
dat.drop("item_condition", axis="columns", inplace=True)

In [34]:
dat.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 54115 entries, 0 to 60108
Data columns (total 10 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   brand                     54115 non-null  object 
 1   fuel_type                 54115 non-null  object 
 2   model_date                54115 non-null  float64
 3   vehicle_transmission      54115 non-null  object 
 4   Car_name                  54115 non-null  object 
 5   Year                      54115 non-null  object 
 6   Location                  54115 non-null  object 
 7   mileage_from_odometer_km  54115 non-null  float64
 8   vehicle_engine_cc         54115 non-null  float64
 9   price_usd                 54115 non-null  float64
dtypes: float64(4), object(6)
memory usage: 4.5+ MB


In [35]:
#save new df
dat.to_csv("used_car_dataset_cleaned.csv", index=False)